## 8. トレーニング済みモデルの推論テスト

In [ ]:
def generate_response(model, tokenizer, prompt, max_length=100):
    """モデルを使用してレスポンスを生成"""
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    
    if torch.cuda.is_available():
        inputs = inputs.to("cuda")
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=max_length,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # プロンプト部分を除去
    if generated_text.startswith(prompt):
        generated_text = generated_text[len(prompt):].strip()
    
    return generated_text

# テスト用プロンプト
test_prompts = [
    "次の文章を要約してください: 機械学習は人工知能の重要な分野です",
    "以下の質問に答えてください: プログラミングの基本とは何ですか？",
    "次のコードを説明してください: print('Hello World')",
]

print("🧪 Testing DPO-trained model...")
print("=" * 60)

for i, prompt in enumerate(test_prompts, 1):
    response = generate_response(model, tokenizer, prompt)
    print(f"\nTest {i}:")
    print(f"Prompt: {prompt}")
    print(f"Response: {response}")
    print("-" * 40)

## 9. 結果の可視化とまとめ

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# トレーニング履歴の可視化
if hasattr(dpo_trainer.state, 'log_history'):
    log_history = dpo_trainer.state.log_history
    
    # ロスの推移を抽出
    train_losses = []
    eval_losses = []
    steps = []
    
    for log in log_history:
        if 'loss' in log:
            train_losses.append(log['loss'])
            steps.append(log.get('step', len(train_losses)))
        if 'eval_loss' in log:
            eval_losses.append(log['eval_loss'])
    
    # グラフの作成
    plt.figure(figsize=(12, 4))
    
    # 訓練ロス
    plt.subplot(1, 2, 1)
    plt.plot(steps, train_losses, 'b-', label='Training Loss')
    plt.title('Training Loss Over Time')
    plt.xlabel('Steps')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    # 評価ロス
    if eval_losses:
        plt.subplot(1, 2, 2)
        eval_steps = np.linspace(0, max(steps), len(eval_losses))
        plt.plot(eval_steps, eval_losses, 'r-', label='Evaluation Loss')
        plt.title('Evaluation Loss Over Time')
        plt.xlabel('Steps')
        plt.ylabel('Loss')
        plt.legend()
        plt.grid(True)
    
    plt.tight_layout()
    plt.show()

print("\n🎉 DPO Training Completed Successfully!")
print("\n📋 Summary:")
print(f"   • Model: {model_name}")
print(f"   • Training samples: {len(train_dataset)}")
print(f"   • Evaluation samples: {len(eval_dataset)}")
print(f"   • Final training loss: {train_result.training_loss:.4f}")
print(f"   • Trainable parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad):,}")

if torch.cuda.is_available():
    print(f"   • Peak GPU memory: {torch.cuda.max_memory_allocated() / 1e9:.2f} GB")

## 10. モデルの保存とダウンロード

訓練済みモデルをGoogle Driveに保存したり、ローカルにダウンロードできます。

In [ ]:
# Google Driveにマウント（オプション）
if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # Google Driveにモデルをコピー
    import shutil
    drive_path = '/content/drive/MyDrive/dpo_model'
    
    print(f"📁 Copying model to Google Drive: {drive_path}")
    shutil.copytree('./final_dpo_model', drive_path, dirs_exist_ok=True)
    print("✅ Model saved to Google Drive!")

# ファイルサイズの確認
import os
def get_folder_size(folder_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(folder_path):
        for filename in filenames:
            filepath = os.path.join(dirpath, filename)
            total_size += os.path.getsize(filepath)
    return total_size

model_size = get_folder_size('./final_dpo_model')
print(f"\n📊 Final model size: {model_size / 1e6:.1f} MB")

# ファイル一覧
print("\n📁 Model files:")
for root, dirs, files in os.walk('./final_dpo_model'):
    for file in files:
        file_path = os.path.join(root, file)
        file_size = os.path.getsize(file_path) / 1e6
        print(f"   {file}: {file_size:.1f} MB")

# DPO Training on Google Colab - 直接選好最適化

このノートブックはGoogle Colabで実行可能なDPO（Direct Preference Optimization）トレーニングコードです。

## 特徴
- 🚀 Google Colab GPU対応
- 📊 リアルタイム学習監視
- 💾 自動チェックポイント保存
- 🎯 効率的なLoRA微調整

## システム要件
- Google Colab (GPU推奨)
- 約2-3GB GPU メモリ
- インターネット接続

## 1. Google Colab 環境設定と依存関係インストール

In [ ]:
import os
import sys
import torch
import pandas as pd
import json
from pathlib import Path

# プロジェクトルートを追加
project_root = Path().absolute().parent
sys.path.append(str(project_root))

print(f"Project root: {project_root}")
print(f"Current working directory: {os.getcwd()}")

# Google Colabでの実行確認
import sys
import os

# Colabかどうかを確認
IN_COLAB = 'google.colab' in sys.modules
print(f"Running in Google Colab: {IN_COLAB}")

# 必要なパッケージをインストール
if IN_COLAB:
    print("📦 Installing required packages...")
    !pip install transformers[torch] trl peft datasets accelerate bitsandbytes evaluate
    !pip install --upgrade torch torchvision torchaudio
    print("✅ Package installation completed!")

# GPUメモリをクリア
import torch
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"🔥 CUDA is available! Device: {torch.cuda.get_device_name()}")
    print(f"💾 GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️ CUDA not available, using CPU")

In [ ]:
# デバイス設定
if torch.cuda.is_available():
    device = torch.device("cuda")
    gpu_name = torch.cuda.get_device_name()
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"🚀 Using GPU: {gpu_name}")
    print(f"💾 GPU Memory: {gpu_memory:.1f} GB")
else:
    device = torch.device("cpu")
    print("⚠️ GPU not available, using CPU")

print(f"Device: {device}")
print(f"PyTorch version: {torch.__version__}")

# メモリ使用量を確認
if torch.cuda.is_available():
    print(f"Current GPU memory usage: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

## 2. DPOデータセットの生成

Colabでは外部ファイルがないため、データセットをその場で生成します。

In [ ]:
import json
import pandas as pd
from typing import List, Dict
import random

def generate_dpo_dataset(num_samples: int = 500) -> List[Dict]:
    """DPOトレーニング用のサンプルデータセットを生成"""
    
    # 様々なタスクのテンプレート
    tasks = [
        {
            "prompt": "次の文章を要約してください: {text}",
            "chosen": "この文章は{topic}について述べており、主なポイントは{point}です。",
            "rejected": "文章の内容は複雑で、様々な要素が含まれています。",
        },
        {
            "prompt": "以下の質問に答えてください: {question}",
            "chosen": "{answer}について説明します。{detail}",
            "rejected": "その質問は難しいですね。様々な観点があります。",
        },
        {
            "prompt": "次のコードを説明してください: {code}",
            "chosen": "このコードは{function}を実行します。{explanation}",
            "rejected": "コードの動作は複雑で、詳細な分析が必要です。",
        },
    ]
    
    topics = ["AI技術", "プログラミング", "データサイエンス", "機械学習", "ウェブ開発"]
    questions = ["AIとは何ですか？", "機械学習の種類は？", "データ分析の手順は？"]
    codes = ["def hello(): print('Hello')", "x = [1,2,3]; print(sum(x))", "import pandas as pd"]
    
    dataset = []
    for i in range(num_samples):
        task = random.choice(tasks)
        
        # プロンプトの生成
        if "{text}" in task["prompt"]:
            topic = random.choice(topics)
            prompt = task["prompt"].format(text=f"{topic}に関する詳細な説明")
            chosen = task["chosen"].format(topic=topic, point=f"{topic}の重要性")
        elif "{question}" in task["prompt"]:
            question = random.choice(questions)
            prompt = task["prompt"].format(question=question)
            chosen = task["chosen"].format(answer="AI", detail="人工知能は計算機による知的な処理です")
        else:
            code = random.choice(codes)
            prompt = task["prompt"].format(code=code)
            chosen = task["chosen"].format(function="関数定義", explanation="指定された処理を実行します")
        
        dataset.append({
            "prompt": prompt,
            "chosen": chosen,
            "rejected": task["rejected"]
        })
    
    return dataset

# データセット生成
print("🔄 Generating DPO dataset...")
dpo_data = generate_dpo_dataset(1000)
df = pd.DataFrame(dpo_data)

print(f"✅ Generated {len(df)} samples")
print(f"Columns: {list(df.columns)}")
print("\n📊 Sample data:")
df.head(2)

In [ ]:
# データセット統計
print("=== Dataset Statistics ===")
print(f"Total samples: {len(df)}")
print(f"Average prompt length: {df['prompt'].str.len().mean():.1f} chars")
print(f"Average chosen length: {df['chosen'].str.len().mean():.1f} chars")
print(f"Average rejected length: {df['rejected'].str.len().mean():.1f} chars")

print("\n=== Sample Example ===")
sample = df.iloc[0]
print(f"Prompt: {sample['prompt']}")
print(f"Chosen: {sample['chosen']}")
print(f"Rejected: {sample['rejected']}")

## 3. モデルとトークナイザーの設定

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model, TaskType

# Colab GPU環境に適した軽量モデル
model_name = "microsoft/DialoGPT-small"  # 約117M parameters

print(f"📥 Loading model: {model_name}")

# トークナイザーの読み込み
tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print("✅ Tokenizer loaded successfully")
print(f"Vocabulary size: {len(tokenizer)}")

In [ ]:
# モデルの読み込み（GPU最適化）
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto" if torch.cuda.is_available() else None,
    low_cpu_mem_usage=True,
)

print("✅ Base model loaded successfully")

# パラメータ数の確認
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params:,}")
print(f"Model size: ~{total_params * 2 / 1e6:.1f} MB (fp16)")

# GPUメモリ使用量
if torch.cuda.is_available():
    print(f"GPU memory after model loading: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

## 4. LoRA (Low-Rank Adaptation) 設定

効率的な微調整のためのLoRA設定

In [ ]:
# LoRA設定 (Colab環境最適化)
lora_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,  # Low rank for memory efficiency
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["c_attn", "c_proj"],  # DialoGPT specific modules
    bias="none",
)

# LoRAモデルの作成
model = get_peft_model(model, lora_config)

# 訓練可能パラメータの確認
model.print_trainable_parameters()

# GPU使用量の確認
if torch.cuda.is_available():
    print(f"GPU memory after LoRA: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

## 5. データ前処理とDPOトレーニング準備

In [ ]:
from datasets import Dataset
from trl import DPOTrainer, DPOConfig

# HuggingFace Dataset形式に変換
dataset = Dataset.from_pandas(df)

# 訓練・検証分割
dataset_split = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = dataset_split['train']
eval_dataset = dataset_split['test']

print(f"📊 Training samples: {len(train_dataset)}")
print(f"📊 Evaluation samples: {len(eval_dataset)}")

# サンプルデータの確認
print("\n🔍 Sample training data:")
print(train_dataset[0])

## 6. DPO (Direct Preference Optimization) トレーニング実行

In [ ]:
def generate_ad_copy(prompt, max_length=150):
    """広告コピーを生成する関数"""
    inputs = tokenizer.encode(prompt, return_tensors="pt")
    
    if torch.backends.mps.is_available():
        inputs = inputs.to("mps")
    
    with torch.no_grad():
        outputs = model.generate(
            inputs,
            max_length=max_length,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
        )
    
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # プロンプト部分を除去
    if generated_text.startswith(prompt):
        generated_text = generated_text[len(prompt):].strip()
    
    return generated_text

# テスト実行
test_prompt = "【テーマ】雨の日でもワクワクするニュースアプリを紹介してください"
result = generate_ad_copy(test_prompt)

print(f"プロンプト: {test_prompt}")
print(f"生成結果: {result}")

# DPOトレーニング設定
training_args = DPOConfig(
    output_dir="./dpo_results",
    num_train_epochs=1,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=5e-6,
    logging_steps=10,
    eval_steps=50,
    save_steps=100,
    evaluation_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    dataloader_num_workers=0,
    remove_unused_columns=False,
    bf16=False,  # Disable for compatibility
    fp16=torch.cuda.is_available(),
    gradient_checkpointing=True,
    max_length=128,
    max_prompt_length=64,
)

print("⚙️ DPO training configuration created")

## 7. DPOトレーナーの初期化と実行

実際のDPOトレーニングは `scripts/train_dpo.py` を使用して実行します。

ターミナルで以下のコマンドを実行してください：

```bash
cd /path/to/AD_Tech_SLM
python scripts/train_dpo.py
```

## 8. モデル評価

トレーニング後のモデルを評価するセクション

# DPOトレーナーの初期化
dpo_trainer = DPOTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    beta=0.1,  # DPO temperature parameter
)

print("🚀 DPO Trainer initialized")

# GPUメモリ使用量の確認
if torch.cuda.is_available():
    print(f"GPU memory before training: {torch.cuda.memory_allocated() / 1e9:.2f} GB")

# トレーニング開始
print("\n🔥 Starting DPO training...")
print("This may take 10-20 minutes depending on your GPU")

train_result = dpo_trainer.train()

print("\n✅ Training completed!")
print(f"Final train loss: {train_result.training_loss:.4f}")

In [ ]:
# トレーニング済みモデルを読み込んで評価する場合
# このセクションはトレーニング完了後に実行

test_prompts = [
    "【テーマ】健康管理をサポートするフィットネスアプリを紹介してください",
    "【テーマ】料理初心者向けのレシピアプリを紹介してください",
    "【テーマ】読書好きのための電子書籍アプリを紹介してください",
]

print("=== 生成テスト ===")
for i, prompt in enumerate(test_prompts, 1):
    result = generate_ad_copy(prompt)
    print(f"\nテスト {i}:")
    print(f"プロンプト: {prompt}")
    print(f"生成結果: {result}")
    print("-" * 50)

# トレーニング後の評価
print("📊 Evaluating trained model...")
eval_results = dpo_trainer.evaluate()

print("\n📈 Evaluation Results:")
for key, value in eval_results.items():
    if isinstance(value, float):
        print(f"  {key}: {value:.4f}")
    else:
        print(f"  {key}: {value}")

# モデルの保存
print("\n💾 Saving trained model...")
dpo_trainer.save_model("./final_dpo_model")
tokenizer.save_pretrained("./final_dpo_model")

print("✅ Model saved successfully!")